In [1]:
import pandas
import json
from os import listdir
from os.path import isfile, join

In [2]:
data_dir = '../dataset/raw/JupyterNotebook'

In [3]:
def get_files(dir_path):
    return [join(dir_path, f) for f in listdir(dir_path) if isfile(join(dir_path, f))]

In [4]:
def read_json_file(file_path):
    try:
        return json.load(open(f))
    except json.decoder.JSONDecodeError:
        return None

In [5]:
data = []
'''
data = [
    [{}, {},..., {}], # issue
    [{}, {},..., {}]
]
'''
for f in get_files(data_dir):
    issue = read_json_file(f)
    if issue:
        data.append(read_json_file(f))

In [6]:
# Removing empty txts
# data = [d for d in data if d]

In [7]:
data[0]

[{'url': 'https://api.github.com/repos/jupyter/notebook/issues/comments/824259486',
  'html_url': 'https://github.com/jupyter/notebook/issues/6040#issuecomment-824259486',
  'issue_url': 'https://api.github.com/repos/jupyter/notebook/issues/6040',
  'id': 824259486,
  'node_id': 'MDEyOklzc3VlQ29tbWVudDgyNDI1OTQ4Ng==',
  'user': {'login': 'kevin-bates',
   'id': 22599560,
   'node_id': 'MDQ6VXNlcjIyNTk5NTYw',
   'avatar_url': 'https://avatars.githubusercontent.com/u/22599560?v=4',
   'gravatar_id': '',
   'url': 'https://api.github.com/users/kevin-bates',
   'html_url': 'https://github.com/kevin-bates',
   'followers_url': 'https://api.github.com/users/kevin-bates/followers',
   'following_url': 'https://api.github.com/users/kevin-bates/following{/other_user}',
   'gists_url': 'https://api.github.com/users/kevin-bates/gists{/gist_id}',
   'starred_url': 'https://api.github.com/users/kevin-bates/starred{/owner}{/repo}',
   'subscriptions_url': 'https://api.github.com/users/kevin-bates/su

In [11]:
issue_count = 0
data_required = []
for issue in data:
    for comment in issue:
        comment_ind = {}
        comment_ind['issue_count'] = issue_count # to group them to belong to same issue
        comment_ind['id'] = comment['id']
        comment_ind['created_at'] = comment['created_at']
        comment_ind['updated_at'] = comment['updated_at']
        comment_ind['author_association'] = comment['author_association']
        comment_ind['body'] = comment['body']
        comment_ind['user_login'] = comment['user']['login']
        comment_ind['user_type'] = comment['user']['type']
        comment_ind['user_site_admin'] = comment['user']['site_admin']
        data_required.append(comment_ind)
    issue_count += 1

In [12]:
data_df = pandas.DataFrame(data_required)

In [13]:
data_df

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
0,0,824259486,2021-04-21T18:13:22Z,2021-04-21T18:13:22Z,MEMBER,This is a known issue in [jupyter_client](http...,kevin-bates,User,False
1,1,325328641,2017-08-28T11:29:17Z,2017-08-28T11:29:17Z,MEMBER,Thanks - actually heading cells are no longer ...,takluyver,User,False
2,1,325383558,2017-08-28T15:17:48Z,2017-08-28T15:17:48Z,CONTRIBUTOR,Alright. Fixing it now...,syedtshah,User,False
3,1,325409403,2017-08-28T16:51:12Z,2017-08-28T16:51:12Z,MEMBER,Thanks :-),takluyver,User,False
4,2,175818892,2016-01-27T19:44:25Z,2016-01-27T19:44:25Z,MEMBER,This is an iPython bug from what I can tell. I...,jakirkham,User,False
...,...,...,...,...,...,...,...,...,...
30729,4748,526858512,2019-08-31T19:17:41Z,2019-08-31T19:17:41Z,CONTRIBUTOR,@kevin-bates since nobody has reviewed this ye...,benthayer,User,False
30730,4748,526862244,2019-08-31T20:14:56Z,2019-08-31T20:14:56Z,MEMBER,Hi Ben - sorry for the frustration. It certai...,kevin-bates,User,False
30731,4748,526864628,2019-08-31T20:55:56Z,2019-08-31T20:55:56Z,CONTRIBUTOR,@marijnh I see you've been active on the CodeM...,benthayer,User,False
30732,4748,527335475,2019-09-03T07:14:14Z,2019-09-03T07:14:14Z,NONE,"@bthayer2365 Hi. Sorry, too busy to start revi...",marijnh,User,False


In [15]:
data_df.user_site_admin.unique()

array([False,  True])

In [19]:
data_df.user_type.unique()

array(['User', 'Bot'], dtype=object)

In [21]:
data_df.groupby('user_type')['id'].nunique() # 67 bot messages. Need to be cleaned?

user_type
Bot        67
User    30667
Name: id, dtype: int64

In [26]:
data_df.groupby('user_site_admin')['id'].nunique() # 10 Admin messages. What are they?

user_site_admin
False    30724
True        10
Name: id, dtype: int64

In [30]:
data_df[data_df['user_site_admin'] == True]

,issue_count,id,created_at,updated_at,author_association,body,user_login,user_type,user_site_admin
8458,1246,380271464,2018-11-13T15:10:18Z,2021-04-04T00:21:31Z,NONE,Observed behavior: Users periodically open not...,glortho,User,True
9856,1246,252996107,2017-08-25T19:24:10Z,2021-04-04T00:22:43Z,CONTRIBUTOR,Addresses issue #1749 \r\n\r\nHad to add some ...,itsJiaqi,User,True
10078,1246,236846207,2017-06-19T10:27:31Z,2021-04-06T00:16:43Z,NONE,Converting a cell to markdown (via the drop do...,jhelie,User,True
16475,2019,439063661,2018-11-15T14:44:07Z,2018-11-15T19:52:41Z,NONE,We've confirmed that this does appear to happe...,glortho,User,True
16476,2019,439075797,2018-11-15T15:17:42Z,2018-11-15T15:18:09Z,NONE,I could be off-base but it seems like we'd eit...,glortho,User,True
16478,2019,439094745,2018-11-15T16:09:31Z,2018-11-15T16:09:31Z,NONE,Thank you kindly for your attention to this @k...,glortho,User,True
16481,2019,439260107,2018-11-16T02:24:40Z,2018-11-16T02:24:40Z,NONE,"Awesome Kevin!\n\nOn Thu, Nov 15, 2018 at 20:0...",glortho,User,True
20009,2653,309690427,2017-06-20T08:59:40Z,2017-06-20T08:59:40Z,NONE,Closing this as the issue seems to have disapp...,jhelie,User,True
20011,2653,310130539,2017-06-21T16:17:35Z,2017-06-21T16:17:35Z,NONE,@mpacer I'm on 5.0.0 and can't readily downgra...,jhelie,User,True
20436,2746,420352074,2018-09-11T17:21:28Z,2018-09-11T17:21:28Z,NONE,We get this error periodically when first navi...,glortho,User,True


In [31]:
data_df.author_association.unique()

array(['MEMBER', 'CONTRIBUTOR', 'NONE'], dtype=object)

In [32]:
data_df.groupby('author_association')['id'].nunique()

author_association
CONTRIBUTOR     5312
MEMBER         12820
NONE           12602
Name: id, dtype: int64